In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

food = pd.read_csv("/Users/viktorijakesaite/Desktop/CAMBRIDGE POSTDOC/UPF/data files/foodleveldietarydatauk_new.csv", encoding='latin1')

In [14]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    They weights are in effect first normalized so that they 
    sum to 1 (and so they must not all be 0).

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, np.sqrt(variance), len (values))

Removing missing values: person day missing observations

In [15]:
def list_wo_nan (l):
    result_0 = []
    result_1 = []
    for y in l:
        if y[1] == y[1]:
            result_0.append(y[0])
            result_1.append(y[1])
        else:
            print ('NaN')
    return result_0, result_1

In [16]:
food2 = pd.read_csv ("/Users/viktorijakesaite/Desktop/CAMBRIDGE POSTDOC/UPF/reduced_dataset.csv", encoding='latin1')

Merge food level and individual level datasets

In [17]:
food_final = food.merge(food2, left_on='seriali', right_on='seriali', how = 'left')

In [18]:
food_final.head(5)

,seriali,SurveyYear,Age,Sex,Country,diarymth,DayofWeek,DayNo,DiaryDaysCompleted,MealTimeDescription,...,wtr_Y78,wtn_Y78,wtb_Y78,wtsu_Y78,GOR,wti_Y9,wti_UKY1234rs,wti_Y56rs,wti_Y78rs,wti_Y911rs
0,40322051,NDNS Year 4,60.0,2,Scotland,February,Friday,1,4,5pm to 7:59pm,...,NaN,NaN,NaN,NaN,NaN,NaN,0.159795,NaN,NaN,NaN
1,10513211,NDNS Year 1,50.0,1,Scotland,September,Sunday,4,4,12 noon to 1:59pm,...,NaN,NaN,NaN,NaN,NaN,NaN,0.345726,NaN,NaN,NaN
2,50308182,NDNS Year 5,1.0,1,England,NaN,Wednesday,1,4,2pm to 4:59pm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.423533,NaN,NaN
3,50308182,NDNS Year 5,1.0,1,England,NaN,Wednesday,1,4,2pm to 4:59pm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.423533,NaN,NaN
4,900603021,NDNS Year 9,NaN,1,England,NaN,Thursday,4,4,2pm to 4:59pm,...,NaN,NaN,NaN,NaN,South West,1.160257,NaN,NaN,NaN,1.435215


Exclude alcohol and supplements

In [19]:
categories= {'47A', '47B','48A', '48B', '48C', '49A', '49B', '49C', '49D', '49E', '54A', '54B', '54C', '54D', '54E', '54F', '54G', '54H', '54I', '54J', '54K', '54L', '54M', '54N', '54P'}
food_noalcnovit=food_final[~food_final['SubFoodGroupCode'].isin(categories)]

Create vegetable and fruit purees -- based on 'FoodName' variable

In [20]:
veg_pureed={'BROCOLLI SPEARS FRESH BOILED PUREED','CABBAGE-WINTER KALE FRESH BOILED (PUREED)',
            'CARROT OLD RAW PUREED','CARROTS OLD, FRESH, BOILED PUREED', 'CARROTS YOUNG FRESH BOILED PUREED', 'CARROTS YOUNG FRESH RAW PUREED',  
            'CAULIFLOWER-FRESH BOILED (PUREED)', 'CELERIAC FRESH BOILED (PUREED)', 'CELERY FRESH BOILED PUREED',  'CELERY, FRESH RAW (PUREED)',
            'COURGETTE-BOILED (PUREED)','GARLIC PUREE','GUACAMOLE PURCHASED', 'GUACAMOLE REDUCED FAT PURCHASED',
            'HARICOT BEANS, NO ADDED SALT, CANNED AND DRAINED, COOKED (PUREED)','HUMMUS, NOT CANNED', 'HUMMUS/HOUMOUS, LOW/REDUCED FAT',
            'LEEKS FRESH BOILED (PUREED)', 'LENTILS SPLIT BOILED (PUREED)', 'MUSHROOMS STEWED OR GRILLED (PUREED)',
            'ONIONS BOILED PUREED', 'PEAS FROZEN BOILED (PUREED) FF PROJECT', 'PEAS SPLIT DRIED BOILED (PUREED)', 
            'PEAS-CHICK, NO ADDED SALT OR SUGAR, CANNED AND DRAINED , COOKED (PUREED)', 'PEPPERS GREEN BOILED (PUREED)', 
            'PEPPERS YELLOW FRESH BOILED (PUREED)',  'PUMPKIN, PUREED, BOILED', 'SPINACH RAW NOT BABY SPINACH (PUREED)', 'SPINACH FRESH BOILED (PUREED)', 
            'SWEDE BOILED (PUREED)', 'SWEETCORN, CANNED, DRAINED, NON ADDED SUGAR OR SALT (PUREED) FS PROJECT', 'TURNIPS-BOILED (PUREED)'}

In [21]:
fruit_pureed={'GOJI BERRIES / WOLFBERRIES DRIED (PUREED)','MELON PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS','STRAWBERRIES STEWED WITHOUT SUGAR (PUREED)', 
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C 0-10MG)', 
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C 10-20MG)',
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C OVER 20MG)', 'GRAPEFRUIT RAW, PUREED, FLESH ONLY NO PEEL OR PIPS', 
              'KIWI PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS', 'NECTARINE PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS', 
              'PASSION FRUIT RAW FLESH & SEEDS ONLY (PUREED)', 'RASPBERRY PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS'}

In [22]:
food_noalcnovit.loc[:, 'veg_pureed']  = 0
food_noalcnovit.loc[:, 'fruit_pureed']  = 0
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(veg_pureed), 'veg_pureed'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(veg_pureed), 'TotalGrams']
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(fruit_pureed), 'fruit_pureed'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(fruit_pureed), 'TotalGrams']

Calculate vegetable variable

In [23]:
food_noalcnovit.loc [:, "vegetable"] = food_noalcnovit["Tomatoesg"]+ food_noalcnovit["Brassicaceaeg"]+food_noalcnovit["YellowRedGreeng"]+food_noalcnovit["Beansg"]+food_noalcnovit["OtherVegg"]

Calculate fruit variable

In [24]:
food_noalcnovit.loc [:, "Fruitjuice_per_100_g"] = food_noalcnovit["FruitJuiceg"]/food_noalcnovit["TotalGrams"]*100

Calculate fruit, veg, and nuts per 100g 

In [25]:
food_noalcnovit.loc [:,"fvn_per_100_g"] = (food_noalcnovit["Fruitg"] + food_noalcnovit["vegetable"]+ food_noalcnovit["Nutsg"]+ food_noalcnovit['TomatoPureeg']*2+ food_noalcnovit['veg_pureed']*2+ food_noalcnovit['fruit_pureed']*2+food_noalcnovit['DriedFruitg']*2)/(food_noalcnovit["TotalGrams"]+ food_noalcnovit['TomatoPureeg'] + food_noalcnovit['veg_pureed']+ food_noalcnovit['fruit_pureed']+ food_noalcnovit['DriedFruitg'])*100

Calculate the amount per 100g

In [26]:
food_noalcnovit.loc [:, "Fruitjuice_per_100_g"] = food_noalcnovit["FruitJuiceg"]/food_noalcnovit["TotalGrams"]*100

In [27]:
food_noalcnovit.loc [:, "Smoothies_per_100_g"] = food_noalcnovit["SmoothieFruitg"]/food_noalcnovit["TotalGrams"]*100

In [28]:
food_noalcnovit.loc [:, "EnergykJ_per_100_g"] = food_noalcnovit["EnergykJ"]/food_noalcnovit["TotalGrams"]*100

In [29]:
food_noalcnovit.loc [:, "Saturatedfattyacidsg_per_100_g"] = food_noalcnovit["Saturatedfattyacidsg"]/food_noalcnovit["TotalGrams"]*100

In [30]:
food_noalcnovit.loc [:, "Totalsugarsg_per_100_g"] = food_noalcnovit["Totalsugarsg"]/food_noalcnovit["TotalGrams"]*100

In [31]:
food_noalcnovit.loc [:, "Sodiummg_per_100_g"] = food_noalcnovit["Sodiummg"]/food_noalcnovit["TotalGrams"]*100

In [32]:
food_noalcnovit.loc [:, "AOACFibreg_per_100_g"] = food_noalcnovit["AOACFibreg"]/food_noalcnovit["TotalGrams"]*100

In [33]:
food_noalcnovit.loc [:, "Proteing_per_100_g"] = food_noalcnovit["Proteing"]/food_noalcnovit["TotalGrams"]*100

In [34]:
food_noalcnovit.NOVANumNEW_agreement.value_counts(normalize=True)*100

Unprocessed or minimally processed foods    49.447770
Ultra-processed food and drink products     36.375105
Processed culinary ingredients               8.073888
Processed foods                              6.103237
Name: NOVANumNEW_agreement, dtype: float64

Create a binary variable equalling one if UPF and zero otherwise

In [35]:
food_noalcnovit.loc[:, 'NOVA_UPF_NUPF_boolean'] = (food_noalcnovit['NOVANumNEW_agreement'] == 'Ultra-processed food and drink products')

Classify HFSS foods using NDNS food database based on the NPM 2004/5 version 

A POINTS

In [36]:
energy_points = {0: [-1, 335], 1: [335, 670], 2: [670, 1005], 3: [1005, 1340], 
                 4: [1340, 1675], 5: [1675, 2010], 6: [2010, 2345], 7: [2345, 2680],
                 8: [2680, 3015], 9: [3015, 3350], 10: [3350]}
food_noalcnovit.loc [:, 'energy_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in energy_points:
    interval = energy_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['EnergykJ_per_100_g'] > interval[0]) & (food_noalcnovit['EnergykJ_per_100_g'] <= interval[1]),'energy_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['EnergykJ_per_100_g'] > interval[0]), 'energy_a_points'] = points

In [37]:
satfat_points = {0: [-1, 1], 1: [1, 2], 2: [2, 3], 3: [3, 4], 
                 4: [4, 5], 5: [5, 6], 6: [6, 7], 7: [7, 8],
                 8: [8, 9], 9: [9, 10], 10: [10]}
food_noalcnovit.loc[:, 'satfat_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in satfat_points:
    interval = satfat_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Saturatedfattyacidsg_per_100_g'] > interval[0]) & (food_noalcnovit['Saturatedfattyacidsg_per_100_g'] <= interval[1]),'satfat_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Saturatedfattyacidsg_per_100_g'] > interval[0]), 'satfat_a_points'] = points

In [38]:
tsugar_points = {0: [-1, 4.5], 1: [4.5, 9], 2: [9, 13.5], 3: [13.5, 18], 
                 4: [18, 22.5], 5: [22.5, 27], 6: [27, 31], 7: [31, 36],
                 8: [36, 40], 9: [40, 45], 10: [45]}
food_noalcnovit.loc[:, 'tsugar_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in tsugar_points:
    interval = tsugar_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Totalsugarsg_per_100_g'] > interval[0]) & (food_noalcnovit['Totalsugarsg_per_100_g'] <= interval[1]),'tsugar_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Totalsugarsg_per_100_g'] > interval[0]), 'tsugar_a_points'] = points

In [39]:
sodium_points = {0: [-1, 90], 1: [90, 180], 2: [180, 270], 3: [270, 360], 
                 4: [360, 450], 5: [450, 540], 6: [540, 630], 7: [630, 720],
                 8: [720, 810], 9: [810, 900], 10: [900]}
food_noalcnovit.loc[:, 'sodium_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in sodium_points:
    interval = sodium_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Sodiummg_per_100_g'] > interval[0]) & (food_noalcnovit['Sodiummg_per_100_g'] <= interval[1]),'sodium_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Sodiummg_per_100_g'] > interval[0]), 'sodium_a_points'] = points

Calculate total A points

In [40]:
food_noalcnovit.loc[:, 'total_a_points'] = food_noalcnovit["energy_a_points"] + food_noalcnovit["satfat_a_points"] + food_noalcnovit["tsugar_a_points"] + food_noalcnovit["sodium_a_points"] 

C POINTS

In [41]:
smoothies = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]}
food_noalcnovit.loc [:, 'smoothies'] = np.array(len(food_noalcnovit) * [0])
for points in smoothies:
    interval = smoothies[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Smoothies_per_100_g'] > interval[0]) & (food_noalcnovit['Smoothies_per_100_g'] <= interval[1]),'smoothies'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Smoothies_per_100_g'] > interval[0]), 'smoothies'] = points

In [42]:
fruit_juices = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]}
food_noalcnovit.loc [:, 'fruit_juice'] = np.array(len(food_noalcnovit) * [0])
for points in fruit_juices:
    interval = fruit_juices[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Fruitjuice_per_100_g'] > interval[0]) & (food_noalcnovit['Fruitjuice_per_100_g'] <= interval[1]),'fruit_juice'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Fruitjuice_per_100_g'] > interval[0]), 'fruit_juice'] = points

In [43]:
fvn_points = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]} 
food_noalcnovit.loc[:, 'fvn_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in fvn_points:
    interval = fvn_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['fvn_per_100_g'] > interval[0]) & (food_noalcnovit['fvn_per_100_g'] <= interval[1]),'fvn_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['fvn_per_100_g'] > interval[0]), 'fvn_c_points'] = points

In [44]:
aoacfibre_points = {0: [-1, 0.9], 1: [0.9, 1.9], 2: [1.9, 2.8], 3: [2.8, 3.7], 4: [3.7, 4.7], 5: [4.7]} 

food_noalcnovit.loc[:, 'aoacfibre_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in aoacfibre_points:
    interval = aoacfibre_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['AOACFibreg_per_100_g'] > interval[0]) & (food_noalcnovit['AOACFibreg_per_100_g'] <= interval[1]),'aoacfibre_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['AOACFibreg_per_100_g'] > interval[0]), 'aoacfibre_c_points'] = points

If a food or drink scores 11 or more ‘A’ points then it cannot score points for protein unless it also scores 5 points for fruit, vegetables and nuts.

In [45]:
protein_points = {0: [-1, 1.6], 1: [1.6, 3.2], 2: [3.2, 4.8], 3: [4.8, 6.4], 4: [6.4, 8.0], 5: [8.0]} 

food_noalcnovit.loc[:, 'protein_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in protein_points:
    interval = protein_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0]) & (food_noalcnovit['Proteing_per_100_g'] <= interval[1])  & (food_noalcnovit['total_a_points'] < 11),'protein_c_points'] = points
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0]) & (food_noalcnovit['Proteing_per_100_g'] <= interval[1])  & (food_noalcnovit['total_a_points'] >= 11) & (food_noalcnovit['fvn_c_points'] >= 5),'protein_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0])  & (food_noalcnovit['total_a_points'] < 11), 'protein_c_points'] = points
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0])  & (food_noalcnovit['total_a_points'] >= 11) & (food_noalcnovit['fvn_c_points'] >= 5), 'protein_c_points'] = points

Calculate total C POINTS

In [46]:
food_noalcnovit.loc[:, 'total_c_points'] = food_noalcnovit["fvn_c_points"] + food_noalcnovit["aoacfibre_c_points"] + food_noalcnovit["protein_c_points"] +food_noalcnovit['fruit_juice']+ food_noalcnovit['smoothies']

TOTAL POINTS

In [47]:
food_noalcnovit.loc[:, "total_points"] = food_noalcnovit["total_a_points"]-food_noalcnovit ["total_c_points"]

In [48]:
food_noalcnovit.loc[:, "lesshealthy"] = food_noalcnovit["total_points"] >= 4 

Create a drink set - include all drink names (Remove fruit juice and smoothies since they are calculated elsewhere)

In [49]:
drink_set={'SEMI SKIMMED MILK','SKIMMED MILK', 'SOFT DRINKS NOT LOW CALORIE', 'SOFT DRINKS LOW CALORIE', 'COMMERCIAL TODDLERS FOODS AND DRINKS', 
       '1% Fat Milk', 'TEA COFFEE AND WATER', 'WHOLE MILK'} 

In [50]:
drink_set2={'ACTIMEL PROBIOTIC DRINKING YOGURT',
'ACTIMEL PROBIOTIC YOGURT DRINK 0.1% FAT',
'BENECOL PLUS HEART YOGURT DRINK', 'BENECOL YOGURT DRINKS',
'BENECOL YOGURT DRINKS, CONTAINING STANOLS',
'CHILDRENS YOGURT DRINK WITH OMEGA 3, CALCIUM AND VITAMIN D',
'CHILDRENS YOGURT DRINK WITH VITAMIN D',
'MULLER VITALITY PROBIOTIC DRINK',
'MULLER VITALITY PROBIOTIC DRINK WITH OMEGA 3',
'OPTIFIT YOGURT DRINKS, ANY FLAVOUR, FORTIFIED WITH VITS C, E, B6',
'SMOOTHIES WITH FRUIT AND DAIRY PRODUCTS, BOTTLED, PURCHASED, NOT FORTIFIED',
'YAKULT', 'YOGURT DRINK', 'YOGURT DRINK CONTAINING FRUIT PUREE',
'YOGURT DRINK FORTIFIED WITH VITAMINS B6, C AND D'}

In [51]:
food_noalcnovit.loc[food_noalcnovit['MainFoodGroupDesc'].isin(drink_set), 'lesshealthy'] = food_noalcnovit.loc[food_noalcnovit['MainFoodGroupDesc'].isin(drink_set), "total_points"]>= 1 

In [52]:
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(drink_set2), 'lesshealthy'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(drink_set2), "total_points"]>= 1 

In [53]:
np.sum(food_noalcnovit["lesshealthy"])/len(food_noalcnovit["lesshealthy"])

0.33401006155548796

In [54]:
np.sum(food_noalcnovit["NOVA_UPF_NUPF_boolean"])/len(food_noalcnovit["NOVA_UPF_NUPF_boolean"])

0.3637510482667603

In [55]:
food_noalcnovit_upf = food_noalcnovit[food_noalcnovit['NOVA_UPF_NUPF_boolean']]

Check the proportion of HFSS if added artificial sweeteners and soft drinks, low calorie

In [56]:
food_noalcnovit_upf.loc[:,"LH_assd"] = (food_noalcnovit_upf["lesshealthy"])| (food_noalcnovit_upf['MainFoodGroupDesc'] == "ARTIFICIAL SWEETENERS")|(food_noalcnovit_upf['MainFoodGroupDesc'] == "SOFT DRINKS LOW CALORIE")

Check the proportion of HFSS if added artificial sweeteners and soft drinks, low calorie

In [57]:
np.sum(food_noalcnovit_upf["LH_assd"] )/len(food_noalcnovit_upf.loc[:,"LH_assd"] )

0.6673027981928571

Proportion of energy derived from HFSS+ARTIFICIAL SWEETENERS AND SOFT DRINKS, LOW CALORIE

In [58]:
np.sum(food_noalcnovit_upf["LH_assd"]* food_noalcnovit_upf['Energykcal'])/np.sum(food_noalcnovit_upf['Energykcal'])

0.5899550903027799

Proportion of energy derived from HFSS among UPF sample

In [59]:
np.sum((food_noalcnovit_upf["lesshealthy"]* food_noalcnovit_upf['Energykcal']))/np.sum(food_noalcnovit_upf['Energykcal'])#len(food_noalcnovit[food_noalcnovit["NOVA_UPF_NUPF_boolean"]])

0.5884218011082163

Proportion of weight derived from HFSS among UPF sample

In [60]:
np.sum((food_noalcnovit_upf["lesshealthy"]* food_noalcnovit_upf['TotalGrams']))/np.sum(food_noalcnovit_upf['TotalGrams'])#len(food_noalcnovit[food_noalcnovit["NOVA_UPF_NUPF_boolean"]])

0.3846324171490843

Summarise for all foods

In [61]:
value_notupf_healthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_upf_healthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_notupf_lesshealthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_upf_lesshealthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.16110810710781281
0.16110810710781281
0.2026429411589475
0.2026429411589475


Create a sum of weights based on the following (df['c'] = df['a'] + df['b']). It's been advised not to use 'apply() for numeric operations' 
https://stackoverflow.com/questions/26886653/create-new-column-based-on-values-from-other-columns-apply-a-function-of-multi

In [62]:
food_noalcnovit.loc[food_noalcnovit['wti_UKY1234rs'] != food_noalcnovit['wti_UKY1234rs'], 'wti_UKY1234rs'] = 0
food_noalcnovit.loc[food_noalcnovit['wti_Y56rs'] != food_noalcnovit['wti_Y56rs'], 'wti_Y56rs'] = 0
food_noalcnovit.loc[food_noalcnovit['wti_Y78rs'] != food_noalcnovit['wti_Y78rs'], 'wti_Y78rs'] = 0
food_noalcnovit.loc[food_noalcnovit['wti_Y911rs'] != food_noalcnovit['wti_Y911rs'], 'wti_Y911rs'] = 0

In [63]:
food_noalcnovit.loc[:, 'total_weights'] = food_noalcnovit ['wti_UKY1234rs']+ food_noalcnovit['wti_Y56rs'] + food_noalcnovit['wti_Y78rs'] + food_noalcnovit['wti_Y911rs'] 

SUMMARIES FOR INDIVIDUAL DAILY CONSUMPTION, WEIGHTED

The food diary entries are based on 3 or 4 day entry. 

HFSS WEIGHTED 

In [64]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4], if we want to limit to 4 days only. 

Grouping data by serial number and day number. Proportion of HFSS by total weight  

In [65]:
grouped_food_noalcnovit_individual = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [66]:
grouped_food_noalcnovit_individual = grouped_food_noalcnovit_individual.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [67]:
x = grouped_food_noalcnovit_individual
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.14965527358618932, 0.10857883584192428, 15655)

UPF WEIGHTED 

In [68]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [69]:
grouped_food_noalcnovit_individual = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [70]:
grouped_food_noalcnovit_individual = grouped_food_noalcnovit_individual.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [71]:
x = grouped_food_noalcnovit_individual
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.2969718505879666, 0.1825537416263026, 15655)

upf anf hfss weighted by weight 

In [72]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [73]:
grouped_food_noalcnovit_upf_hfss = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [74]:
grouped_food_noalcnovit_upf_hfss = grouped_food_noalcnovit_upf_hfss.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [75]:
        
x = grouped_food_noalcnovit_upf_hfss
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.11508037813580918, 0.09997822601381627, 15655)

UPF only

In [76]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [77]:
grouped_food_noalcnovit_upf_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [78]:
grouped_food_noalcnovit_upf_only = grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [79]:
x = grouped_food_noalcnovit_upf_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.18189147245215737, 0.1340645545207261, 15655)

HFSS ONLY

In [80]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [81]:
grouped_food_noalcnovit_hfss_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean']* x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [82]:
grouped_food_noalcnovit_hfss_only = grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [83]:
x = grouped_food_noalcnovit_hfss_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.18189147245215737, 0.1340645545207261, 15655)

Neither upf nor hfss

In [84]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [85]:
grouped_food_noalcnovit_hfss_upf_neither = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean']* x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [86]:
grouped_food_noalcnovit_hfss_upf_neither = grouped_food_noalcnovit_hfss_upf_neither.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [87]:
x = grouped_food_noalcnovit_hfss_upf_neither
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.6684532539616534, 0.18608018054552883, 15655)

UPF SAMPLE ONLY

In [88]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [89]:
grouped_food_noalcnovit_hfss_upfONLY = food_noalcnovit_four_upf.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [90]:
grouped_food_noalcnovit_hfss_upfONLY = grouped_food_noalcnovit_hfss_upfONLY.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [91]:
x = grouped_food_noalcnovit_hfss_upfONLY
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.40046305965829343, 0.1901057047182845, 15654)

INDIVIDUAL DAILY WEIGHTED BY ENERGY

We have some missing observations, which are removed. 

HFSS 

In [92]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [93]:
grouped_food_noalcnovit_individual_HFSS = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [94]:
grouped_food_noalcnovit_individual_HFSS = grouped_food_noalcnovit_individual_HFSS.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [95]:
x = grouped_food_noalcnovit_individual_HFSS
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.4366413046609531, 0.13212178702808752, 15632)

UPF 

In [96]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [97]:
grouped_food_noalcnovit_individual_UPF = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [98]:
grouped_food_noalcnovit_individual_UPF = grouped_food_noalcnovit_individual_UPF.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [99]:
x = grouped_food_noalcnovit_individual_UPF
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.5491282608364579, 0.1570218502142123, 15632)

hfss and upf

In [100]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [101]:
grouped_food_noalcnovit_hfss_upf = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['Energykcal'])/np.sum(x['Energykcal'])))

In [102]:
grouped_food_noalcnovit_hfss_upf = grouped_food_noalcnovit_hfss_upf.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [103]:
x = grouped_food_noalcnovit_hfss_upf
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.3028426114408593, 0.13755814847629347, 15632)

Less healthy only

In [104]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [105]:
grouped_food_noalcnovit_hfss_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean']* x['Energykcal'])/np.sum(x['Energykcal'])))

In [106]:
grouped_food_noalcnovit_hfss_only = grouped_food_noalcnovit_hfss_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [107]:
x = grouped_food_noalcnovit_hfss_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.1337986932200938, 0.08667656472097669, 15632)

In [108]:

x = grouped_food_noalcnovit_hfss_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.1337986932200938, 0.08667656472097669, 15632)

UPF ONLY

In [109]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [110]:
grouped_food_noalcnovit_upf_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['Energykcal'])/np.sum(x['Energykcal'])))

In [111]:
grouped_food_noalcnovit_upf_only = grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [112]:
x = grouped_food_noalcnovit_upf_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.24628564939559863, 0.10465561048609152, 15632)

neither upf nor hfss

In [113]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [114]:
grouped_food_noalcnovit_upf_hfss_neither = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean']* x['Energykcal'])/np.sum(x['Energykcal'])))

In [115]:
grouped_food_noalcnovit_upf_hfss_neither = grouped_food_noalcnovit_upf_hfss_neither.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [116]:
x = grouped_food_noalcnovit_upf_hfss_neither
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.3170730459434484, 0.136177616590026, 15632)

UPF SAMPLE ONLY

In [117]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [118]:
grouped_food_noalcnovit_upfsample = food_noalcnovit_four_upf.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean']* x['Energykcal'])/np.sum(x['Energykcal'])))

In [119]:
grouped_food_noalcnovit_upfsample = grouped_food_noalcnovit_upfsample.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [120]:
x = grouped_food_noalcnovit_upfsample
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


(0.538583668996331, 0.1725595060605059, 15649)

Summaries for individual daily consumption

individual daily consumption

In [121]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [122]:
grouped_food_noalcnovit_individual = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'])/len(x['lesshealthy'])))

In [123]:
grouped_food_noalcnovit_individual = grouped_food_noalcnovit_individual.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [124]:
grouped_food_noalcnovit_individual

seriali
10101032                  (1.0634205, 0.609375)
10101042      (0.93880826, 0.26329139610389607)
10101111       (1.8103184, 0.47808228300219147)
10101151        (2.1780238, 0.2347550675675676)
10101161      (0.86775178, 0.20160566084479126)
                            ...                
901212091     (0.50142938, 0.22903505512201164)
901212101      (0.34787661, 0.3892615290578906)
901212113     (0.045170452, 0.4791666666666667)
901212132      (0.41703701, 0.3233743686868687)
901212133    (0.044426363, 0.34119152046783624)
Length: 15655, dtype: object

In [125]:
x = grouped_food_noalcnovit_individual
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.32042224117714335, 0.1063430981406811, 15655)

for UPFs

In [126]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [127]:
grouped_food_noalcnovit_individual_upf = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [128]:
grouped_food_noalcnovit_individual_upf = grouped_food_noalcnovit_individual_upf.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [129]:
x = grouped_food_noalcnovit_individual_upf
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.33558241141438844, 0.1558754514402835, 15655)

UPFs and HFSS

In [130]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [131]:
grouped_food_noalcnovit_upf_hfss = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [132]:
grouped_food_noalcnovit_upf_hfss = grouped_food_noalcnovit_upf_hfss.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [133]:
x = grouped_food_noalcnovit_upf_hfss
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.18532702511829816, 0.10027081217398329, 15655)

UPF only

In [134]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [135]:
grouped_food_noalcnovit_upf_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [136]:
grouped_food_noalcnovit_upf_only = grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [137]:
x = grouped_food_noalcnovit_upf_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.1502553862960903, 0.0886796082185862, 15655)

HFSS only

In [138]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [139]:
grouped_food_noalcnovit_hfss_only = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [140]:
grouped_food_noalcnovit_hfss_only = grouped_food_noalcnovit_hfss_only.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [141]:
x = grouped_food_noalcnovit_hfss_only
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.13509521605884517, 0.08065166842957708, 15655)

Neither HFSS nor UPF

In [142]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [143]:
grouped_food_noalcnovit_hfss_upf_neither = food_noalcnovit.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [144]:
grouped_food_noalcnovit_hfss_upf_neither = grouped_food_noalcnovit_hfss_upf_neither.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [145]:
x = grouped_food_noalcnovit_hfss_upf_neither
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.5293223725267663, 0.14361502766662748, 15655)

UPF and HFSS OVERLAP AMONG UPF PRODUCTS

In [146]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [147]:
grouped_food_noalcnovit_hfss_upf_upfsample = food_noalcnovit_four_upf.groupby(['seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [148]:
grouped_food_noalcnovit_hfss_upf_upfsample = grouped_food_noalcnovit_hfss_upf_upfsample.groupby(level=[0]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [149]:
x = grouped_food_noalcnovit_hfss_upf_upfsample
weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]])

(0.5512048969219875, 0.1550869313692123, 15654)

Individual analyses by gender

Less healthy

In [150]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [151]:
grouped_food_noalcnovit_individual = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'])/len(x['lesshealthy'])))

In [152]:
grouped_food_noalcnovit_individual = grouped_food_noalcnovit_individual.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [153]:
result = grouped_food_noalcnovit_individual.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print (result.to_frame().to_string())

                                                    0
Sex                                                  
1    (0.33248007384476963, 0.10611370726986862, 7207)
2     (0.3087528565373516, 0.10525748481486889, 8448)


UPF 

In [154]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [155]:
grouped_food_noalcnovit_individual_upf = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [156]:
grouped_food_noalcnovit_individual_upf = grouped_food_noalcnovit_individual_upf.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [157]:

        
grouped_food_noalcnovit_individual_upf.groupby(level=[0]).apply (lambda x:  weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.34790595962317916, 0.15865962867024272, 7207)
2    (0.32365587136863866, 0.15218550203457257, 8448)
dtype: object

UPF and HFSS

In [158]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [159]:
grouped_food_noalcnovit_hfss_upf= food_noalcnovit.groupby(['Sex','seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [160]:
grouped_food_noalcnovit_hfss_upf_combined = grouped_food_noalcnovit_hfss_upf.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [161]:
grouped_food_noalcnovit_hfss_upf_combined.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.19348154849575872, 0.10360183205633607, 7207)
2    (0.17743520305799965, 0.09628278479741917, 8448)
dtype: object

UPF only

In [162]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [163]:
grouped_food_noalcnovit_upf_only= food_noalcnovit.groupby(['Sex','seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [164]:
grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1       (0.1543877274605864, 0.109088462089645, 28693)
2    (0.14607743353722027, 0.10809008983962169, 33604)
dtype: object

HFSS only

In [165]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [166]:
grouped_food_noalcnovit_hfss_only= food_noalcnovit.groupby(['Sex','seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [167]:
grouped_food_noalcnovit_hfss_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.13896995553819408, 0.09710415326102818, 28693)
2     (0.1313086983843479, 0.09689329994884956, 33604)
dtype: object

Neither HFSS nor UPF

In [168]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [169]:
grouped_food_noalcnovit_hfss_upf_neither= food_noalcnovit.groupby(['Sex','seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [170]:
grouped_food_noalcnovit_hfss_upf_neither.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.5132231607701108, 0.16841807157468827, 28693)
2    (0.5451024273574165, 0.16525495339710378, 33604)
dtype: object

UPF and HFSS among UPF only

In [171]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [172]:
grouped_food_noalcnovit_upf_sample = food_noalcnovit_four_upf.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'])/len(x['NOVA_UPF_NUPF_boolean'])))

In [173]:
grouped_food_noalcnovit_upf_sample.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1     (0.5520878247770055, 0.2115957015557635, 28610)
2    (0.5513435694771694, 0.21606286230923785, 33504)
dtype: object

Less healthy by total weight

In [174]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [175]:
grouped_food_noalcnovit_hfss_weight = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [176]:
grouped_food_noalcnovit_hfss_weight = grouped_food_noalcnovit_hfss_weight.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [177]:
grouped_food_noalcnovit_hfss_weight.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1     (0.1634696426571547, 0.11186906960389853, 7207)
2    (0.13628594008416187, 0.10355663722383324, 8448)
dtype: object

UPF by total weight 

In [178]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [179]:
grouped_food_noalcnovit_upf_weight = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [180]:
grouped_food_noalcnovit_upf_weight.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.31684780918853217, 0.21524940899270767, 28693)
2     (0.2775920940968645, 0.20729302755958556, 33604)
dtype: object

UPF and less healthy by weight

In [181]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [182]:
grouped_food_noalcnovit_upf_hfss = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [183]:
grouped_food_noalcnovit_upf_hfss.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.12645067825187906, 0.13273978287794938, 28693)
2    (0.10406973554676126, 0.11834909741169834, 33604)
dtype: object

Less healthy only

In [184]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [185]:
grouped_food_noalcnovit_hfss_only = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [186]:
grouped_food_noalcnovit_hfss_only = grouped_food_noalcnovit_hfss_only.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [187]:
grouped_food_noalcnovit_hfss_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1      (0.0369787492805659, 0.04510721242862614, 7207)
2    (0.03224848275785799, 0.040347273560757985, 8448)
dtype: object

UPF only

In [188]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [189]:
grouped_food_noalcnovit_upf_only = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [190]:
grouped_food_noalcnovit_upf_only = grouped_food_noalcnovit_upf_only.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [191]:
grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.19040804673234296, 0.1336211983499595, 7207)
2    (0.1736492631075482, 0.13397773015356185, 8448)
dtype: object

Neither HFSS nor UPF

In [192]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [193]:
grouped_food_noalcnovit_neither_hfss_upf = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [194]:
grouped_food_noalcnovit_neither_hfss_upf = grouped_food_noalcnovit_neither_hfss_upf.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [195]:
grouped_food_noalcnovit_neither_hfss_upf.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.6461223106105024, 0.18644171155452244, 7207)
2      (0.69006479680829, 0.18315520513233108, 8448)
dtype: object

UPF and less healthy for UPF products only

In [196]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [197]:
grouped_food_noalcnovit_upf_sample = food_noalcnovit_four_upf.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [198]:
grouped_food_noalcnovit_upf_sample = grouped_food_noalcnovit_upf_sample.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [199]:
grouped_food_noalcnovit_upf_sample.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.40472688697985537, 0.18976112875300488, 7206)
2     (0.3963371190362525, 0.19034763815712427, 8448)
dtype: object

Neither HFSS nor UPF

In [200]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [201]:
grouped_food_noalcnovit_neither_upf_hfss = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'] * x['TotalGrams'])/np.sum(x['TotalGrams'])))

In [202]:
grouped_food_noalcnovit_neither_upf_hfss = grouped_food_noalcnovit_neither_upf_hfss.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [203]:
grouped_food_noalcnovit_neither_upf_hfss.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

Sex
1    (0.6461223106105024, 0.18644171155452244, 7207)
2      (0.69006479680829, 0.18315520513233108, 8448)
dtype: object

UPF only

In [204]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [205]:
grouped_food_noalcnovit_upf_only = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [206]:
grouped_food_noalcnovit_upf_only = grouped_food_noalcnovit_upf_only.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [207]:
grouped_food_noalcnovit_upf_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1     (0.2546428038012798, 0.10346476723729686, 7199)
2    (0.23819824451648564, 0.10516483292941904, 8433)
dtype: object

healthy only 

In [208]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [209]:
grouped_food_noalcnovit_healthy_only = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [210]:
grouped_food_noalcnovit_healthy_only = grouped_food_noalcnovit_healthy_only.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [211]:
grouped_food_noalcnovit_healthy_only.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1    (0.13262808331478868, 0.08551424681410798, 7199)
2    (0.13493151855753963, 0.08777185127661415, 8433)
dtype: object

Less healthy and upf weighted for UPF goods only

In [212]:
food_noalcnovit_four_upf = food_noalcnovit[(food_noalcnovit["NOVA_UPF_NUPF_boolean"])]

In [213]:
grouped_food_noalcnovit_upf_sample = food_noalcnovit_four_upf.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [214]:
grouped_food_noalcnovit_upf_sample = grouped_food_noalcnovit_upf_sample.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [215]:
grouped_food_noalcnovit_upf_sample.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1     (0.5325703845923553, 0.1717586418214759, 7205)
2    (0.5444044605600549, 0.17313238969215491, 8444)
dtype: object

NEITHER LESS HEALTHY NOR UPF WEIGHTED BY ENERGY

In [216]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [217]:
grouped_food_noalcnovit_upf_hfss_NEITHER = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (~x['lesshealthy'] * ~x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [218]:
grouped_food_noalcnovit_upf_hfss_NEITHER = grouped_food_noalcnovit_upf_hfss_NEITHER.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [219]:
grouped_food_noalcnovit_upf_hfss_NEITHER.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1      (0.3035763766884603, 0.1346661088553221, 7199)
2    (0.33013407407407175, 0.13635850138946892, 8433)
dtype: object

Less healthy and upf weighted

In [220]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [221]:
grouped_food_noalcnovit_upf_hfss = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [222]:
grouped_food_noalcnovit_upf_hfss = grouped_food_noalcnovit_upf_hfss.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [223]:
        grouped_food_noalcnovit_upf_hfss.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1    (0.30915273619547123, 0.14068966754049822, 7199)
2      (0.296736162851903, 0.13417603915050544, 8433)
dtype: object

UPF by gender weighted

In [224]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [225]:
grouped_food_noalcnovit_upf = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['NOVA_UPF_NUPF_boolean'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [226]:
grouped_food_noalcnovit_upf = grouped_food_noalcnovit_upf.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [227]:
grouped_food_noalcnovit_upf.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights = [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1     (0.563795539996751, 0.15719356121282152, 7199)
2    (0.5349344073683887, 0.15554419961137503, 8433)
dtype: object

Less healthy by gender weighted

In [228]:
#food_noalcnovit_four = food_noalcnovit[food_noalcnovit['DiaryDaysCompleted']==4]

In [229]:
grouped_food_noalcnovit_hfss = food_noalcnovit.groupby(['Sex', 'seriali', 'DayNo']).apply (lambda x: (np.min(x['total_weights']), np.sum (x['lesshealthy'] * x['Energykcal'])/np.sum(x['Energykcal'])))

In [230]:
grouped_food_noalcnovit_hfss

Sex  seriali    DayNo
1    10101151   1          (2.1780238, 0.5493650779652315)
                2          (2.1780238, 0.4414979382116011)
                3          (2.1780238, 0.5853923449896691)
                4          (2.1780238, 0.4559027168324364)
     10102011   1          (2.5626781, 0.4574927769681163)
                                       ...                
2    901212091  4        (0.50142938, 0.05258746842253459)
     901212101  1         (0.34787661, 0.4633904190989554)
                2         (0.34787661, 0.4869438509921944)
                3        (0.34787661, 0.42032910619226493)
                4        (0.34787661, 0.36186893881925053)
Length: 62297, dtype: object

In [231]:
grouped_food_noalcnovit_hfss = grouped_food_noalcnovit_hfss.groupby(level=[0, 1]).apply (lambda x: (np.min([y[0] for y in x]), np.mean ([y[1] for y in x])))

In [232]:
grouped_food_noalcnovit_hfss 

Sex  seriali  
1    10101151       (2.1780238, 0.5080395194997346)
     10102011        (2.5626781, 0.593120781392033)
     10102032      (0.37143102, 0.5713907058041552)
     10102042      (0.65639466, 0.4233047856990973)
     10102071       (1.9840486, 0.5410280017863974)
                                ...                
2    901210163     (0.22372176, 0.4137829229386466)
     901210264    (0.096313551, 0.3444355746779695)
     901211241    (0.83734238, 0.29269165840465416)
     901212091    (0.50142938, 0.20915363809200288)
     901212101     (0.34787661, 0.4331330787756663)
Length: 15655, dtype: object

In [233]:

    grouped_food_noalcnovit_hfss.groupby(level=[0]).apply (lambda x: weighted_avg_and_std (list(list_wo_nan(x)[1]), weights =  [y for y in list_wo_nan(x)[0]]))

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


Sex
1    (0.4417808195102599, 0.13247951525865165, 7199)
2     (0.4316676814094426, 0.1315836903570158, 8433)
dtype: object